### Suivit du tuto blitz de Pytorch pour définir un classifier simple: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

### Loader 

#### Défintion des classes

In [ ]:
classes = ('0', '1', '2')
# 0=Vimizer /// 1=youtube /// 2=tiktok 

### Bibliothèques

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from video_dataset import  VideoFrameDataset, ImglistToTensor
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid


#### Fonctions utiles 

In [ ]:
####### Affichage des images #############
def plot_video(rows, cols, frame_list, plot_width, plot_height, title: str):
    fig = plt.figure(figsize=(plot_width, plot_height)) # Initialise la figure
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(rows, cols),  # creates 2x2 grid of axes
                     axes_pad=0.3,  # pad between axes in inch.
                     )

    for index, (ax, im) in enumerate(zip(grid, frame_list)):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.set_title(index)
    plt.suptitle(title)
    plt.show()

##### Image preprocessing #####
preprocess = transforms.Compose([
        ImglistToTensor(),  # list of PIL images to (FRAMES x CHANNELS x HEIGHT x WIDTH) tensor
        transforms.Resize(576),  # image batch, resize smaller edge to 576
        transforms.CenterCrop(576),  # image batch, center crop to square 576x576
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        
])

In [ ]:
####### Parametres ######
batch_size = 2 #(inf ou = 2)
####### Chemins ########
import os 
videos_root_train= os.path.join(os.getcwd(), 'dataset_train')
annotation_file_train = os.path.join(videos_root_train, 'annotation.txt')

videos_root_test= os.path.join(os.getcwd(), 'dataset_test')
annotation_file_test = os.path.join(videos_root_test, 'annotation_test.txt')



#### Loader entrainement 

In [ ]:

trainset = VideoFrameDataset(
        root_path=videos_root_train,
        annotationfile_path=annotation_file_train,
        num_segments=5, #Nombre de segment par vidéo
        frames_per_segment=1, #Nombre de frame extraite par segment ( soit 5*1 Frames par vidéo)
        imagefile_template='img_{:05d}.png',
        transform=preprocess,
        test_mode=False # If True, frames are taken from the center of each segment, instead of a random location in each segment.        
    )

print ( 'Nombre de vidéos chargées :' , len(trainset)) 
# Résultat attendu 3 * 9 = 27 vidéos

### Inspectons l'une d'elle numéro 3 ### 
sample = trainset[2] # Liste contenant : (0) le tensor , (1) le label d'une video
frame_tensor = sample[0]  # tensor of shape (NUM_SEGMENTS*FRAMES_PER_SEGMENT) x CHANNELS x HEIGHT x WIDTH
label = sample[1]  # integer label
print('Video Tensor Size:', frame_tensor.size())
print('Label ', label)

trainloader  = torch.utils.data.DataLoader(
        dataset=trainset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

#### Loader de Test

In [ ]:
testset = VideoFrameDataset(
        root_path=videos_root_test,
        annotationfile_path=annotation_file_test,
        num_segments=5, #Nombre de segment par vidéo
        frames_per_segment=1, #Nombre de frame extraite par segment ( soit 5*1 Frames par vidéo)
        imagefile_template='img_{:05d}.png',
        transform=preprocess,
        test_mode=False # If True, frames are taken from the center of each segment, instead of a random location in each segment.        
    )

print ( 'Nombre de vidéos chargées :' , len(testset)) 
# Résultat attendu 3 * 2 = 6 vidéos

testloader  = torch.utils.data.DataLoader(
        dataset=testset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )


#### Definition de l'architecture de notre Classifier

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        ##################### La fonction init permet d'initialiser les couches du réseau ######################
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6,5) # Couche de convolution de 3 canaux en entré, 6 canaux de sortie et un kernet de 5*5
        self.pool = nn.MaxPool2d(2, 2, 6)  # Couche de Pooling 2*2 avec un Stride (pas) de 2
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # Couche Fully Connected avec une entré de 1655 et une sortie de 120
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3) # Sortie de 10 passé à 3 car " classes"

        ##################### Définit la phase de forward propagation ######################
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # ici on appelle la première couche de convolution, on applique la fonction d'activation et une couche de pooling
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # aplatit les données en une seule dimension tout en préservant la taille du batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()



#### Définition des paramètres et métriques 

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
epochs = 2

#### Entrainement du modèle 

In [ ]:

for epoch in range(epochs):  # loop over the dataset multiple times

    print('epoch ', epoch)
    running_loss = 0.0 # On initialise à 0 la perte en début d'epoch pour calculer la perte moyenne sur tous les mini-batch

    for i, data in enumerate(trainloader, 0): # i: indice de mini batch et data: contenu du mini batch

        # On récupère l'input et le label en deux variables distinctes
        inputs, labels = data

        # Réinitialise les gradients des paramètres du modèle à zéro
        optimizer.zero_grad()

        # Phase Forward
        outputs = net(inputs)
        # calcule la perte (erreur) entre les prédictions et les étiquettes réelles du mini-batch
        loss = criterion(outputs, labels)
        # effectue la rétropropagation pour calculer les gradients des paramètres par rapport à la perte
        loss.backward()
        # met à jour les poids du réseau en fonction des gradients calculés via l'optimiseur
        optimizer.step()

        # On met à jour la perte 
        running_loss += loss.item()

        # Affiche la perte tous les 2 mini-batches 
        if i % 2 == 1:    
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2:.3f}')
            running_loss = 0.0

print('Finished Training')

#### Sauvegarde du modèle 

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)
# state_dict() est une méthode de la classe nn.Module en PyTorch 
# qui renvoie un dictionnaire contenant les poids et les biais du modèle.

#### Test de Network sur un Minibatches

##### Ici on va faire la prédiction pour une image 

In [ ]:
dataiter = iter(testloader) # Ici on itère sur le dataset de test
images, labels = next(dataiter) # On récupère le prochain mini-batch 
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
### Si on a besoin de recharger le modèle voilà la commande ###
#net = Net()
#net.load_state_dict(torch.load(PATH))

In [ ]:
### Result sur une video test ###
outputs = net(images) # On effectue la forward propagation pour l'ensebles des images du mini-batches (génère des labels prédits)
_, predicted = torch.max(outputs, 1) # Pour chaque image dans le lot on renvoye les valeurs maximales des classes prédictes (-) et les labels associés (predicted)
# On affiche ces labels 
print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(batch_size)))

#### Test de Network sur tout le dataset de Test pour obtenir l'accuracy du modèle sur ce dataset

In [ ]:
# On initialise notre compteur des labels correctement prédit 
correct = 0
total = 0
# Ici nous n'aurons pas besoin de calculer les gradient nous n'utilisons que la forward propagation 
# pour la prédiction des labels 
# Les étapes sont les mêmes que précédement
with torch.no_grad():
    for data in testloader: #  data: contenu du mini batch
        # On récupère l'input et le label en deux variables distinctes
        images, labels = data
        # Forward des images dans le réseau
        outputs = net(images)
        # La classe avec le score le plus élevé est choisit comme label
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0) #incrémente le compteur total par le nombre d'images dans le mini-lot.
        correct += (predicted == labels).sum().item() 
        #compare les prédictions du modèle avec les étiquettes réelles pour calculer le nombre d'images 
        #correctement prédites dans le mini-lot et l'ajoute à correct.

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

#### Test de Network sur tout le dataset de Test pour obtenir l'accuracy du modèle mais sur chacune des classes

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

#### Command Line pour entrainement sur GPU 

In [ ]:
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:
#print(device)


In [ ]:
#net.to(device)

In [ ]:
#inputs, labels = data[0].to(device), data[1].to(device)